Perguntas: 
número de fatalidades tem diminuido com os anos? *procure pelas colunas de fataliade
durante a noite há mais ocorrências (18:00 a 05:00)?
algumas regiões tem mais ocorrências que outras?
Verificar em quantas cidades já aconteceram ocorrências?
alguns operadores têm mais ocorrências que outros?

In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, hour
spark = SparkSession.builder.appName("MeuApp").getOrCreate()

In [3]:
df_ocorrencias = spark.read.option("delimiter", ";").option("header", True).csv("ocorrencia.csv")
df_ocorrencias.show(truncate=False)

24/05/13 09:06:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+---------------+-----------------------------------------------------------+---------------------------+------------------+------------------+----------------------+-------------+------------+--------------------------------------------------+------------------------+--------+---------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
df_municipios = spark.read.option("delimiter", ",").option("header", True).csv("municipios.csv")
df_municipios.show(truncate=False)

+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+
|codigo_ibge|nome               |latitude|longitude|capital|codigo_uf|siafi_id|ddd|fuso_horario     |
+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+
|5200050    |Abadia de Goiás    |-16.7573|-49.4412 |0      |52       |1050    |62 |America/Sao_Paulo|
|3100104    |Abadia dos Dourados|-18.4831|-47.3916 |0      |31       |4001    |34 |America/Sao_Paulo|
|5200100    |Abadiânia          |-16.197 |-48.7057 |0      |52       |9201    |62 |America/Sao_Paulo|
|3100203    |Abaeté             |-19.1551|-45.4444 |0      |31       |4003    |37 |America/Sao_Paulo|
|1500107    |Abaetetuba         |-1.72183|-48.8788 |0      |15       |0401    |91 |America/Sao_Paulo|
|2300101    |Abaiara            |-7.34588|-39.0416 |0      |23       |1301    |88 |America/Sao_Paulo|
|2900108    |Abaíra             |-13.2488|-41.6619 |0      |29       |3301    |77 

In [17]:
# Definir uma função que verifica o período do dia
def periodo_dia(hora_col):
    return F.when(
        hora_col.isNull(), F.lit(None)
    ).when(
        (hora_col >= 18) | (hora_col < 5), F.lit('noite')
    ).otherwise(F.lit('dia'))

In [20]:
# Adicionar a nova coluna 'Periodo_do_Dia' baseada na coluna 'Hora_da_Ocorrencia'
df_ocorrencias = df_ocorrencias.withColumn('Periodo_do_Dia', periodo_dia(F.hour('Hora_da_Ocorrencia')))

# Mostrar o DataFrame resultante
df_ocorrencias = df_ocorrencias.select("Numero_da_Ocorrencia","Classificacao_da_Ocorrencia","Hora_da_Ocorrencia","Periodo_do_Dia")
df_ocorrencias.show(truncate=False)

+--------------------+---------------------------+------------------+--------------+
|Numero_da_Ocorrencia|Classificacao_da_Ocorrencia|Hora_da_Ocorrencia|Periodo_do_Dia|
+--------------------+---------------------------+------------------+--------------+
|25043               |Incidente                  |22:10             |noite         |
|25081               |Incidente                  |17:56             |dia           |
|25503               |Incidente                  |19:02             |noite         |
|25623               |Incidente                  |15:30             |dia           |
|25624               |Incidente                  |15:25             |dia           |
|25625               |Ocorrência Anormal         |14:58             |dia           |
|25626               |Acidente                   |16:00             |dia           |
|25765               |Incidente                  |17:45             |dia           |
|25766               |Incidente                  |17:30          

Pergunta 2:
Durante a noite há mais ocorrências (18:00 a 05:00)?

In [29]:
df_periodoDia = df_ocorrencias.groupBy("Periodo_do_Dia").agg(F.count(F.col("Periodo_do_Dia")).alias("Total"))
df_periodoDia.filter((F.col("Periodo_do_Dia") != "NULL")).show(truncate=False)
#df_periodoDia.show(truncate=False)

+--------------+-----+
|Periodo_do_Dia|Total|
+--------------+-----+
|dia           |5893 |
|noite         |4032 |
+--------------+-----+

